Implement courtship chasing while keeping a specific distance to the other fly using
visual detection of the distance to the other fly

(1) Person establishes two-fly chase environment / scenario

(2) Person implements visual processing of target fly being chased

(3) Person implements motor control of chasing and velocity control with processed visual input

In [1]:
from flygym import Fly, Camera, SingleFlySimulation, Simulation 
import cv2
import numpy as np
import pickle
from tqdm import trange
import flygym as flygym
from flygym.arena import FlatTerrain

from flygym.preprogrammed import all_leg_dofs


In [2]:
fly1 = Fly(name="1", spawn_pos=(0, 0, 0.5))
fly2 = Fly(name="2", spawn_pos=(3, 0, 0.5))

cam = Camera(fly=fly1, camera_id="Animat/camera_left")

arena = FlatTerrain()

run_time = 0.5 #temps de la simulation
timestep = 1e-4 #Temps de chaque etape
actuated_joints = all_leg_dofs

target_num_steps = int(run_time / timestep) #Nombre de pas pendant la simulation

#RANDOM PATH
data_block_2 = np.zeros((len(actuated_joints), target_num_steps)) #joints a chaque etape (ici 0 encore)
output_t_2 = np.arange(target_num_steps) * timestep #timecode de chaque step

# Generate random data for interpolation
np.random.seed(0)  # for reproducibility
input_t_2 = np.linspace(0, run_time, num=10)  # 100 input time points
data_2 = {joint: np.random.rand(10) for joint in actuated_joints}  # On prend 100 points random

for i, joint in enumerate(actuated_joints):
    data_block_2[i, :] = np.interp(output_t_2, input_t_2, data_2[joint]) #On comble les points entre les 100 de base par interpolation

#DATA PATH


sim_params = flygym.Parameters(
    timestep=1e-4, render_mode="saved", render_playspeed=0.2, draw_contacts=True
)
actuated_joints = flygym.preprogrammed.all_leg_dofs

data_path = flygym.get_data_path("flygym", "data")
with open(data_path / "behavior" / "210902_pr_fly1.pkl", "rb") as f:
    data = pickle.load(f)

target_num_steps = int(run_time / sim_params.timestep)
data_block = np.zeros((len(actuated_joints), target_num_steps))
input_t = np.arange(len(data["joint_LFCoxa"])) * data["meta"]["timestep"]
output_t = np.arange(target_num_steps) * sim_params.timestep
for i, dof in enumerate(actuated_joints):
    data_block[i, :] = np.interp(output_t, input_t, data[dof])




In [3]:
sim = Simulation(
    flies=[fly1, fly2],
    cameras=[cam],
    arena=arena
)

obs, info = sim.reset()

for i in trange(target_num_steps):
    joint_pos_2 = data_block_2[:, i]
    joint_pos = data_block[:, i]
    action = {
        "1": {"joints": joint_pos},
        "2": {"joints": joint_pos_2},
    }
    obs, reward, terminated, truncated, info = sim.step(action)
    sim.render()
    cam.save_video("./test.mp4")

  6%|▌         | 289/5000 [00:20<05:31, 14.22it/s]


KeyboardInterrupt: 